In [1]:
import warnings

warnings.filterwarnings("ignore")
import os

if "jbook" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
FORCE = True

# Data Quality Analysis for Aspect-Based Sentiment Analysis with Transformer Models
Data Quality Anomaly Detection (DQAD) and analysis are standard operating procedures in NLP, and for good reason: machine learning models often struggle with performance degrading inconsistencies and irrelevant artifacts in text data. Whereas, these *traditional* models, highly sensitive to noise, require careful preprocessing to function effectively; *transformer* models such as GPT-2 {cite}`BetterLanguageModels`, BERT {cite}`devlinBERTPretrainingDeep2019a`, and its descendants, RoBERTA {cite}`liuRoBERTaRobustlyOptimized2019`, and DeBERTa {cite}`heDeBERTaDecodingenhancedBERT2021` exhibit greater stability and robustness due in part to certain architectural differences: **subword tokenization**, **contextual embeddings**, and **self-attention mechanisms**.

> When timeless principles—universal truths—manifest in their principals, nations and principalities flourish.

Consider the sentence above, 

## Subword Tokenization**
Experiments have demonstrated that subword tokenization methods such as Byte-Pair Encoding {cite}`Gage1994ANA`, WordPiece, and SentencePiece {cite}`kudoSentencePieceSimpleLanguage2018` significantly improve transformer model performance in natural language processing tasks such as machine translation and sentence classification. By breaking down words into [more] frequently-occurring subwords units, models can more effectively deal with rare and out-of-vocabulary words, leading to better overall accuracy and performance. 

- For instance, the word `concensus`, into frequently encountered subwords e.g., [`con`], [`cen`], and [`sus`], models can more accurately that encounter `consensus`   frequency-maximized subwords and characters, models can effectively handle rare and out-of-vocabulary (OOV) words and misspellings {cite}`wollebAssessingImportanceFrequency2024`. By breaking words into frequency-maximized subwords and characters, models learn nuanced relationships   , by breaking them down into smaller subword units.  and improve performance on range of tasks, including. When a model 'sees' the word `consensus`, it may tokenize it as [`con`], [`sen`], and [`sus`] based on the frequencies with which the words such as `connect`, `sent`, and `sustain`, that contain these sequences, appear in the corpus. 

Similarly, an OOV word such as  would be tokenized as  [`con`], [`cen`], and [`sus`]. After tokenization, transformer models convert these subword tokens into **embeddings**, or vectors in a high-dimensional space. From directional distance measures, such as **cosine similarity**, known for its effectiveness in high-dimensional spaces, semantic relationships emerge that relate `consensus` rather than `concensus`.

- **Contextual Embeddings and Attention Mechanisms**: Given a corpus, static tokenization mechanisms such as Word2Vec {cite}`mikolovEfficientEstimationWord2013` are idempotent, fixed, and context-independent. That is, the word `concensus` will always be assigned the same unique vector, regardless of the context in which it appears.  Contextual embeddings, by contrast, are dynamic and context-dependent. Self-attention mechanisms simultaneously, and dynamically determine the importance of each word in a sequence, relative to the other words in the same sequence.   in transformer models dynamically and simultaneously determ


  understand that [`] tokenization yielding  is encountered, a model      is broken down into its most frequently occuring pairs, and merged yielding ['pron', 'unc', 'iat', 'ion']. 

While subword tokenization such as Byte-Pair Encoding {cite}`Gage1994ANA`, WordPiece, and SentencePiece {cite}`kudoSentencePieceSimpleLanguage2018`
Transformer models,  exhibit a degree of robustness to noise, derived from advancements in tokenization, contextual embeddings,  from  XLNet, and T5 Studies have shown that transformer-based models like BERT and RoBERTa exhibit robustness to 

While experiments demonstrate that the performance of BERT, a foundational transformer model, degrades significantly as the level of noise (spelling mistakes) increases, models such as RoBERTa show greater stability and robustness {cite}`baglaNoisyTextData2021`. Does this Prediction accuracy, on single sentence tasks such as sentiment analysis, can 
For instance, studies have shown that the impact of text noise on the performance of transformer-based models varies.   the impact of noise on transformer-based language models While studies have shown BERT, a foundational transformer model, to be demonstrates that its performance degredation increases with   significant performance degredation as noise increases 
While studies have demonstrated BERT, a foundational tranformer model,  has shoStudies have shown While studies have shown BERT's performance under noisy    investigating the impact of noise, specifically spelling mistakes, on performance While transformer-based models such as BERT, GPT-3.5, and T5 exhibit and its cousins, DistilBERT, ALBERT, DeBERTaV3, and RoBERTa, exhibit broad robustness 
Experiments studying the effects of noise (spelling mistakes) in text data on transformer-based language models, such as BERT, BART, RoBERTA, ALBERT, XLNet, and T5 that performance degrades significantly as noise increases 
Studies , and largely  and varied based on 1) the type and severity of noise, 2) the size and diversity of data upon which it was trained, and 3) the task to be performed.



 have demonstrated remarkable robustness to noise on sentiment analysis tasks . , such as BERT and its cousins, DistilBERT, ALBERT, DeBERTaV3, and RoBERTa, trained on **billions of words**, primarily sourced from English Wikipedia (~ 2.5 billion words) and Google's BookCorpus (~800 million words), have consistently demonstrated remarkable robustness to abbreviations, emojis, slang, internet jargon, grammatical errors, informal word forms, and misspellings. Moreover, the literature suggests that the performance effects of various types of perturbations are task-specific. On sentiment analysis, transformer models tend to be nearly invariant to perturbations for   performance  Large and diverse datasets, subword tokenization, and self-attention mechanisms, simultenously applied to entire sequences of data ,  and massive and diverse training data wllow collectively Subword tokenization, and *self-attention* mechanisms, which are applied to all tokens in a sentence simultaneously, allow neural networks to capture the relationships *between*, and the contexts among, words and subwords. 

1. **Misspellings**: Subword tokenization, i.e. Byte Pair Encoding (BPE) and SentencePiece, break misspelled words into recognizable subword tokens.  - once positionally encoded, are fed into a series of self-attention layers which capture dependencies and context between tokens. Consequently, a transformer can distiguish `correct` misspellings from context, i.e. 'he is lying on the couch'.
2. **Slang and Informal Language**: Transformers are trained on large, diverse, real-world datasets, enabling them  to capture  on large and diverse, real-world text including slang and informal expressions, transformers embed these linguistic nuances  models  making them adept at understanding and processing these variations. Subword tokenization decomposes these unconventional words into smaller units that the model has encountered in other contexts, enabling generalization. Additionally, transformers’ vast training data captures the distribution and use of slang, embedding these linguistic nuances effectively. 
3. **Emojis and Special Characters**: Subword tokenization treats emojis and special symbols as unique tokens, preserving their semantic value. The self-attention mechanism allows the model to integrate these elements contextually, understanding their contribution to sentiment or meaning within the text. By attending to the relationships between emojis and surrounding words, the model can interpret and generate text that accurately reflects emotional tone or emphasis. 
4. **Abbreviations and Internet Jargon**: Abbreviations and internet-specific language are broken down into meaningful subword segments, allowing transformers to recognize patterns and relate them to standard language forms. The self-attention mechanism plays a crucial role here by dynamically assigning importance to different parts of the input sequence, enabling the model to understand the intended message despite the use of abbreviations. 
5. **Grammatical Errors and Informal Word Forms**: The self-attention mechanism is a fundamental innovation in transformer models. It enables the model to establish contextual relationships between words regardless of their order or grammatical correctness. By weighing the relevance of each word relative to others, the model captures the overarching meaning even in the presence of syntax errors or informal language structures. This flexibility makes transformers robust to variations that would otherwise disrupt traditional models.

Whereas traditional models struggle with inconsistencies in text and require aggressive preprocessing to address noise, transformer models are not only robust to many types of noise but can also derive meaningful insights from them. Importantly, what may seem like noise to traditional approaches—such as slang, emojis, informal language, or unconventional word forms—can carry valuable contextual information. Overprocessing this 'useful noise' can hinder, rather than enhance, model performance and generalization {cite}`languageandmultimodalailamalabimperialcollegelondonukBetterUnderstandingNoise2021`. By preserving or even embracing this *useful* noise, models become more adaptable and effective in practical applications, demonstrating the nuanced trade-offs in handling linguistic noise.

## Data Quality Anomaly Detection (DQAD)
Therefore, we take a nuanced, task-specific approach to data quality assessment and anomaly detection, isolating and removing only *harmful* noise. We define harmful noise as artifacts that do not carry meaning or distort the intended meaning of the text. To ensure high data quality, we assess and flag observations to support analysis along several dimensions of data quality and governance.

### Validity Dimension
Analysis and modeling depend upon **valid** data that adhere to expected formats, structures, and encoding; as well as, patterns, values, and ranges.

1. **Values**: 
    1. Ensure that `ratings` are discrete values in [1,5] and that all category information is valid.
2. **Structure**: Evaluate the degree to which the data are represented with appropriate data types, and are free of corrupting noise that carries no inherent meaning for modeling. Noise flagged during the data quality anomaly detection stage are:
    1. **Excessive Special Characters**: The presence of excessive or random special characters can corrupt the intended meaning of the text and make it harder for models to interpret context. Accuracy checks ensure that these characters are only present when they add legitimate semantic value, such as in programming-related text or stylized writing.  
    2. **Non-ASCII Characters**: While transformers can process non-ASCII characters, they may introduce unintended complexities or errors, especially when non-ASCII content is mixed into primarily English text without a clear purpose. Accuracy checks flag these occurrences to determine if they are contextually appropriate or represent an error in the data.  
    3. **Control Characters**: Control characters, which are non-printable characters like tabs or line breaks embedded in text data, can disrupt text parsing and processing. Ensuring their absence or appropriate use maintains the structural accuracy needed for smooth NLP operations. 
    4. **HTML Characters**: Text data sourced from the web may contain HTML tags or character entities that interfere with the text's readability and model understanding. Accuracy checks to sanitize or transform these elements to their intended textual form.  
    5. **Excessive Whitespace**: Extra spaces or line breaks, though seemingly minor, can affect text tokenization and representation in models. Normalizing whitespace ensures text is processed in a consistent, meaningful way.  
    6. **Accented and Diacritic Characters**: While accented characters are valid in many languages, their unintended presence in primarily non-accented text can indicate data entry errors. Checks for these characters verify if they are linguistically appropriate or require correction. 
3. **Patterns**: Pattern analysis can expose signals of SPAM, non-sensical sequences of text, and excessive repetition. Data quality anomalies of interest include:
    1. **Repeated Sequences**: Reviews with excessive repetition of sequences, such as repeated letters, words, or patterns, can indicate spam or low-quality content. Validity checks identify such sequences, enabling corrective measures to ensure high-quality input for NLP models. 
    2. **Repeated Words**: Similar to repeated sequences, the presence of redundant words may indicate automated or spam-like content. Detecting and addressing these issues helps maintain the linguistic integrity of the dataset.  
    3. **Repeated Phrases**: Repeated phrases can dilute the semantic richness of the text and may signify low-quality or irrelevant content. Validity checks ensure these phrases are flagged for removal or further examination. 
    4. **Elongation**: Text elongation, like in "sooo coool," is often used to emphasize words but may not be handled uniformly by models. Accuracy checks flag or normalize elongation to ensure consistent semantic interpretation. 
4. **Information**: In the context of language models, low perplexity is another signal of repetitive or predictable patterns that may not carry substantive meaning. Flagging low perplexity reviews ensures that text has appropriate complexity and variability for analysis and modeling.

By incorporating these **validity** checks, we verify that the data adheres to expected structures, norms, and formats, reducing the risk of disruptions during text analysis. 

## Completeness Dimension
> Every valid logical expression is provable. Equivalently, every logical expression is either satisfiable or refutable.  (Gödel 1929). 

Here we refer to the Completeness *Dimension*, not to be confused with Gödel’s Completeness *Theorem* above! (*This is why they make computer scientists take philosophy classes*). Primarily, our focus is NULL values in the dataset. A secondary examination is the degree to which the data are complete in terms of representation along two dimensions: **categories** and **sentiments**. Completeness mitigates against model bias and prediction errors derived from class imbalance and under/over-representation.

## Relevance Dimension
The **Relevance** dimension in text data quality ensures that the content is contextually appropriate and meaningful for the specific NLP task or analysis at hand. In other words, the text must be pertinent to the domain, language, or focus of the project. Relevance checks filter out content that could mislead models or degrade the performance of algorithms by introducing off-topic or linguistically inconsistent information.
1. **Non-English App Names**: In datasets where the primary focus is on English-language content, non-English app names can be a source of confusion or skew analysis results. Relevance checks flag these instances, allowing us to either exclude or process them separately to maintain linguistic consistency. 
2. **Non-English Review Text**: Similar to non-English app names, reviews written in languages other than English may be irrelevant to models trained specifically on English text. Relevance checks identify non-English text, helping ensure the data aligns with the model's language capabilities and task requirements. 
3. **Review Length < 3**: Very short reviews, typically less than three words, often lack substantive information or context. These reviews are unlikely to provide meaningful insights and may act as noise, affecting sentiment analysis or topic modeling performance. Relevance checks filter these short reviews to maintain a focus on text that contributes valuable content to the analysis.

By assessing relevancy, we ensure that the text data are appropriate, meaningful, and aligned with the goals of the analysis. This dimension helps avoid the inclusion of extraneous or off-topic content that could distort model training or analysis results.

## Uniqueness Dimension
The **Uniqueness** dimension examines the degree to which duplication is extant. Duplicated content can skew analysis, reduce the diversity of linguistic features, and lead to misleading insights. Therefore, **uniqueness** crucially influences the integrity and reliability of analytical results and model inference. Here, we evaluate the uniqueness of:
1. **Records**: Duplicate records are flagged for analysis and removal during the data cleaning stage.
2. **Review Id**: Duplicate review identifiers can indicate duplication or corruption of the qualitative and quantitative review content. Uniqueness checks detect and flag duplicate review IDs by date, keeping the most recent record is the source of truth. 
3. **Review Text**: In user-generated text content, some duplication is inherent; however, excessive duplication can reduce diversity and lead to model bias and compromised generalizability. We flag all duplicate reviews for further scrutiny.   

By enforcing the **Uniqueness** dimension, we ensure that analyses are based on a diverse and representative sample of the text.

## Data Governance: Privacy and Compliance
1. **URLs**: Reviews containing URLs may not provide meaningful textual content for analysis and can disrupt language models. Validity checks identify and flag URLs, allowing for their removal or replacement to maintain textual coherence. 
2. **Phone Numbers**: Similar to URLs, phone numbers are often irrelevant to the semantic content of a review and may interfere with text processing. Validity checks detect phone numbers, ensuring that they are either masked or removed to avoid skewing the analysis. 
3. **Email Addresses**: Email addresses can introduce noise and potentially violate privacy policies. Detecting and handling these elements helps maintain data integrity and privacy while ensuring the text remains analyzable. 

Next, we construct and execute the **Data Quality Anomaly Detection** pipeline, adding indicators of data validity, completeness, relevance, uniqueness, and compliance.


## Import Libraries

In [2]:
from discover.container import DiscoverContainer
from discover.infra.config.flow import FlowConfigReader
from discover.core.flow import StageDef
from discover.flow.stage.data_prep.quality import DataQualityStage
from discover.assets.idgen import AssetIDGen
from discover.core.flow import PhaseDef, StageDef

## Dependency Container

In [3]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.flow.stage.base",
    ],
)

## Data Quality Anomaly Detection (DQAD) Pipeline
Following our standard orchestration process, we lodd the configuration using the `FiowConfiguReader`, then construct and execute the **DataQualityDetectionStage** pipeline.

In [4]:
# Obtain the configuration
reader = FlowConfigReader()
stage_config = reader.get_stage_config(phase=PhaseDef.DATAPREP, stage=StageDef.DQD)
# Build and run the stage
stage = DataQualityStage.build(
    stage_config=stage_config, return_dataset=False, force=FORCE
)
dataset = stage.run()

[11/22/2024 02:20:15 AM] [DEBUG] [discover.flow.stage.base.DataQualityStage] [run] : Execution path: RUN
[11/22/2024 02:20:15 AM] [DEBUG] [discover.infra.service.spark.session.SparkSessionPool] [_create_session] : Creating a spark session.
[11/22/2024 02:20:15 AM] [DEBUG] [discover.infra.service.spark.session.SparkSessionPool] [_create_session] : Creating an Spark session. log4j Configuration: file:/home/john/projects/appvocai-discover/log4j.properties




# ============================================================================== #
#                      Data Quality Anomaly Detection Stage                      #
# ============================================================================== #



your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairURLTask
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Fri, 22 Nov 2024 02:20:26
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.16 seconds




                             DetectOrRepairURLTask                              
                             ---------------------                              
                          Start Datetime | Fri, 22 Nov 2024 02:20:26
                       Complete Datetime | Fri, 22 Nov 2024 02:20:27
                                 Runtime | 0.16 seconds


                         DetectOrRepairEmailAddressTask                         
                         ------------------------------                         
                          Start Datetime | Fri, 22 Nov 2024 02:20:27


[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairEmailAddressTask
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.06 seconds
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairPhoneNumberTask
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.05 seconds
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairControlCharsTask
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Fri, 22 Nov 2024 02:20:27
[11/22/2024 02:20:27 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Fri, 22

...

                       Complete Datetime | Fri, 22 Nov 2024 02:20:28
                                 Runtime | 0.05 seconds


                         DetectOrRepairShortReviewsTask                         
                         ------------------------------                         
                          Start Datetime | Fri, 22 Nov 2024 02:20:29
                       Complete Datetime | Fri, 22 Nov 2024 02:20:29
                                 Runtime | 0.04 seconds


                       DetectOrRepairCategoryAnomalyTask                        
                       ---------------------------------                        
                          Start Datetime | Fri, 22 Nov 2024 02:20:29
                       Complete Datetime | Fri, 22 Nov 2024 02:20:29
                                 Runtime | 0.09 seconds


                        DetectOrRepairRatingAnomalyTask                         
                        -------------------------------                     

[11/22/2024 02:20:29 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairRatingAnomalyTask
[11/22/2024 02:20:29 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Fri, 22 Nov 2024 02:20:29
[11/22/2024 02:20:29 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Fri, 22 Nov 2024 02:20:29
[11/22/2024 02:20:29 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.06 seconds


                       Complete Datetime | Fri, 22 Nov 2024 02:20:29
                                 Runtime | 0.06 seconds


[11/22/2024 02:20:32 AM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [_remove_dataset_file_by_filepath] : Removed dataset file at workspace/test/dataset/01_dataprep/appvocai_discover-01_dataprep-04_dqd-review-dataset.parquet from repository.
[11/22/2024 02:20:32 AM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [remove] : Removed dataset dataset-test-dataprep-dqd-review from the repository.
[11/22/2024 02:22:11 AM] [DEBUG] [Stage.run] [wrapper] : Stage: Data Quality Anomaly Detection Stage
[11/22/2024 02:22:11 AM] [DEBUG] [Stage.run] [wrapper] : Stage Started: Fri, 22 Nov 2024 02:20:15
[11/22/2024 02:22:11 AM] [DEBUG] [Stage.run] [wrapper] : Stage Completed: Fri, 22 Nov 2024 02:22:11
[11/22/2024 02:22:11 AM] [DEBUG] [Stage.run] [wrapper] : Stage Runtime: 1.0 minutes and 55.7 seconds




                      Data Quality Anomaly Detection Stage                      
                           Stage Started | Fri, 22 Nov 2024 02:20:15
                         Stage Completed | Fri, 22 Nov 2024 02:22:11
                           Stage Runtime | 1.0 minutes and 55.7 seconds


# ============================================================================ #



With **Data Quality Anomaly Detection** we move on to **Data Quality Analysis (DQA)**.

In [ ]:
repo = container.repo.dataset_repo()
asset_id = AssetIDGen.get_asset_id(
    asset_type="dataset", phase=PhaseDef.DATAPREP, stage=StageDef.DQD, name="review"
)
dataset = repo.get(asset_id=asset_id, distributed=False)

print(dataset)



                            Dataset                             
                           phase | Data Preparation Phase
                           stage | Data Quality Anomaly Detection Stage
                            name | review
                      dt_created | 2024-11-22T02:20:29.220529
                        asset_id | dataset-test-dataprep-dqd-review
                             nlp | False
                     distributed | True
                storage_location | workspace/test/dataset/01_dataprep/appvocai_discover-01_dataprep-04_dqd-review-dataset.parquet
                           nrows | 8670
                           ncols | 34
                            size | 8670
                        consumed | False
                     consumed_by | None
                     dt_consumed | None




In [6]:
dataset.content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8670 entries, 0 to 8669
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       8670 non-null   object        
 1   app_id                                   8670 non-null   object        
 2   app_name                                 8670 non-null   object        
 3   category_id                              8670 non-null   object        
 4   author                                   8670 non-null   object        
 5   rating                                   8670 non-null   int16         
 6   content                                  8670 non-null   object        
 7   vote_sum                                 8670 non-null   int64         
 8   vote_count                               8670 non-null   int64         
 9   date                                     